In [ ]:
import os

if not os.getcwd().endswith("src"):
    %cd ..
%pwd

/home/xqz-u/master/FACT/FACT/src


'/home/xqz-u/master/FACT/FACT/src'

In [ ]:
import implicit
import numpy as np
import pandas as pd
import scipy

import config

/home/xqz-u/py_envs/FACT/lib/python3.10/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
user_artist_df = pd.read_csv(config.LASTFM_DIR / "user_artists.dat", sep="\t")
user_artist_df = user_artist_df.rename(columns={"userID": "user", "artistID": "item"})
user_artist_df

,user,item,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


In [ ]:
# steps:
#     1. keep only top-2500 most listened artists DONE
#     2. pre-process raw counts with log transforms (is it just taking the log?) DONE
#     3. transform into full user-item preference matrix DONE
#     4. split into 70/10/20 train/val/test sets, save the seeds used
#     5. use Implicit library to fit a matrix factorization, using
#        grid-search on hyperparms defined in appendix C.2
#     6. generalize to MovieLens dataset, gpu etc.

In [ ]:
# filter only top k artists
k = 2500
top_k_artists = np.array(user_artist_df.groupby("item")["weight"].sum().sort_values(ascending=False).index)[:k]
user_artist_df = user_artist_df.loc[user_artist_df["item"].isin(top_k_artists)]
assert set(user_artist_df["item"]) == set(top_k_artists)
# log-transform
user_artist_df = user_artist_df.copy() # avoid SettingWithCopy warning
user_artist_df.loc[:, "weight"] = np.log(user_artist_df["weight"])
user_artist_df

,user,item,weight
0,2,51,9.538420
1,2,52,9.366489
2,2,53,9.337061
3,2,54,9.239899
4,2,55,9.103089
...,...,...,...
92795,2100,1276,7.032624
92796,2100,1281,6.350886
92797,2100,2749,6.276643
92798,2100,2765,6.124683


In [ ]:
user_item_df = user_artist_df.pivot(index="user", columns="item", values="weight").fillna(0)
user_item_df

item,2,6,7,8,9,10,12,15,18,19,...,18125,18126,18127,18205,18206,18434,18435,18558,18559,18575
user,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2096,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2097,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# implicit wants sparse matrices (user, item), the docs say (item, user) but they are outdated,
# look at their source code instead
user_item_csr = scipy.sparse.csr_matrix(user_item_df.values)
user_item_csr

<1880x2500 sparse matrix of type '<class 'numpy.float64'>'
	with 67009 stored elements in Compressed Sparse Row format>

In [ ]:
# split into 0.7 train 0.2 val 0.1 test
from implicit import evaluation

seed = 42

train_csr, tmp_csr = evaluation.train_test_split(user_item_csr, train_percentage=0.7, random_state=seed)
val_csr, test_csr = evaluation.train_test_split(tmp_csr, train_percentage=2/3, random_state=seed)

In [ ]:
# initialize a model
# NOTE hyperparameters mentioned in paper:
#     regularization -> "regularization"
#     confidence weight -> is it "alpha"? check Hu, Koren and Volinsky (2008)
#                          or is it bm25 weights as in here
#                          https://github.com/benfred/implicit/blob/871e0c7229b012108131b6211cd617e23a3b24bf/examples/tutorial_lastfm.ipynb
#                          (I don't think so, it should be mentioned in the paper)

import time
from copy import deepcopy

latent_factors = [16, 32, 64, 128]
regularization = [0.01, 0.1, 1.0, 10.0]
alpha = [0.1, 1.0, 10.0, 100.0]

best_model_map = -1.0
best_model_path = config.MODELS_DIR / "lastfm_gt_best_model.npz"
best_model_hyperp_path = config.MODELS_DIR / "lastfm_gt_best_model_hyperp.txt"
best_model_hyperp, best_model = None, None


start = time.time()
for factor in latent_factors:
    for regularizer in regularization:
        for a in alpha:
            model = implicit.als.AlternatingLeastSquares(factors=factor,
                                                         regularization=regularizer,
                                                         alpha=a,
                                                         calculate_training_loss=True)
            model.fit(train_csr)
            map_ = evaluation.ranking_metrics_at_k(model, train_csr, val_csr)["map"]
            if map_ > best_model_map:
                print(f"Best model found! old map: {best_model_map} new map {map_} hyperp: {(factor, regularizer, a)}")
                best_model_map = map_
                best_model = deepcopy(model)
                best_model_hyperp = (factor, regularizer, a)

print(f"Grid search done, took {time.time() - start}s")

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1839 [00:00<?, ?it/s]

Best model found! old map: -1.0 new map 0.08863366859542635 hyperp: (16, 0.01, 0.1)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1839 [00:00<?, ?it/s]

...

  0%|          | 0/1839 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/1839 [00:00<?, ?it/s]

Grid search done, took 526.0281665325165s


TypeError: expected str, bytes or os.PathLike object, not tuple

In [ ]:
best_model.save(best_model_path)
with open(best_model_hyperp_path, "w") as fd:
    fd.write("factor,regularizer,alpha\n")
    fd.write(",".join(list(map(str, best_model_hyperp))))

In [ ]:
print(model.user_factors.shape, model.item_factors.shape)
# NOTE this dot product is basically matrix completion
model.user_factors @ model.item_factors.T

(1880, 50) (2500, 50)


array([[ 0.01825498, -0.08626322, -0.52701044, ...,  0.00844697,
         0.        ,  0.        ],
       [ 0.01688535, -0.01063113, -0.01585248, ...,  0.00751778,
         0.        ,  0.        ],
       [ 0.02635779,  0.00567163, -0.12742268, ..., -0.02085552,
         0.        ,  0.        ],
       ...,
       [ 0.04879622, -0.02720973, -0.03053814, ...,  0.0526298 ,
         0.        ,  0.        ],
       [ 0.07194633, -0.02554585, -0.03692012, ...,  0.00750659,
         0.        ,  0.        ],
       [-0.00216324,  0.18069954, -0.14740887, ...,  0.00135516,
         0.        ,  0.        ]], dtype=float32)

In [ ]:
evaluation.mean_average_precision_at_k(model, train_csr, val_csr)

  0%|          | 0/1839 [00:00<?, ?it/s]

0.12038094186580038